### Approaches to finite state machine design in Python

CS 236 <br>
Fall 2024

Michael A. Goodrich <br>
Brigham Young University <br>
***

### A Note About Python Style

Please use the python style guide linked on Learning Suite under the _Content -> Style Guide_ tab. Since example code and jupyter notebook tutorials will use the style guide, we review the style elements in this tutorial here.

 * __Variable types__. The format for specifying a variable and a type is "name: type = initial_value" as in:
 ```
        my_integer: int = 0
        my string: str = "hello world"
``` 
By contrast, the format that doesn't use the style guide is
```
        my_integer = 0
        my_string = "hello world"
```
* __List, sets, dictionaries__. The format for specifyng a list type is "name: list[type] = initial_value" as in
```
        my_integer_list: list[int] = [1,2,3]
        my_string_integer_dictionary: dict[str,int] = {"a":1, "b":2, "c":3}
        my_string_set: set[str] = set() 
```
By contrast, the format that doesn't use the style guide is
```
        my_integer_list = [1,2,3]
        my_string_integer_dictionary = {"a":1, "b":2, "c":3}
        my_string_set = set() 
```
* __Function argument types__. The format for specifying an argument in a function is "def function_name(argument_name: type)" as in:
```
        def my_function(first_argument: str, second_argument: int)
```
By contrast, the format that doesn't use the style guide is
```
        def my_function(first_argument, second_argument)
```
* __Function return values__. The format for specifying the return type is "def my_function(argument: type) -> return_type:" as in
```
        def my_function(first_argument: str, second_argument: int) -> bool:
```
By contrast, the format that doesn't use the style guide is
```
        def my_function(first_argument, second_argument):
```


___

### What is a Finite State Machine ###

The textbook defines a Finite State Machine (FSM) as a tuple (see Def 3 in section 13.3.3). Think of a _tuple_ as an ordered pair with more than two elements. The tuple for an FSA is $(S,I,O,s_0,f,g)$ where 
* A finite set of states $S$
* A set of input characters $I$
* A set of output characters $O$
* A start state $s_0$
* A transition function $f:S\times I \rightarrow S$
* An output function $g:S\times I \rightarrow O$

Recall from the reading and the lecture that when a function is defined like $f:S\times I \rightarrow S$ the domain of the function is $S\times I$ and the range of the function is $S$. Saying that the domain is the Cartesian product is $S\times I$ means that the input to the function is a tuple $(s,i)$ where $s\in S$ and $i\in I$. In other words, the transition function $f$ maps a _current state_ and an _input_ into a next state yielding $s_{\rm next\_state} = f(s_{\rm current\_state},i)$.

Similarly, the output function $g:S\times I\rightarrow O$ takes maps the tuple of the current state and the input into an output yielding $o = g(s_{\rm current\_state},i)$.

---


We often draw FSMs by representing each state $s\in S$ with a circle, representing the start state by a circle with an arrow pointing toward it, and representing transition functions as arrows connecting one state to another. The labels on the arrow connecting one state to another have two parts: an input $i\in I$ that causes the FSA to transition from one state to another, and the output $o\in O$ that is generated during the transition. 

___



Consider the following finite state machine. 

![cow FSA](cow-fsm.png)

This FSM reads letters from a string one by one. If it reads the string "cow" followed by a new line "\n" then the FSM outputs the number 3. This number represents the number of characters that it has read, minus the new line. It it reads anything else, it outputs the number 0. The greek letter $\lambda$ is called _lambda_. It is used in the figure to indicate that nothing is output. We read $\neg$'c' as _any character except 'c'_. And we read the capital $I$ as any character in the set of possible inputs.

The elements of the FSM tuple for the figure shown above are:
* The finite set of states is $S=\{s_0,s_1,s_2,s_3, s_4,s_{\rm err}\}$
* The finite set of input characters is $I=\{$'$a$'$,$ '$b$', '$c$',$\ldots$, '$y$',  '$z$','\\ n'$\}$
* The finite set of output characters is $O=\{0,1,2,3\}$
* The start state is $s_0$
* The transition function $f:S\times I \rightarrow S$ are all the states connected by the labeled arrows:

| present state | input             | next state |
| :-:           | :-:               | :-:        |
| $s_0$         | 'c'               | $s_1$      |
| $s_0$         | anything but 'c'               | $s_{\rm err}$      |
| $s_1$         | 'o'               | $s_2$      |
| $s_1$         | anything but 'o'               | $s_{\rm err}$      |
| $s_2$         | 'w'               | $s_3$      |
| $s_2$         | anything but 'w'               | $s_{\rm err}$      |
| $s_3$         | '\\n'             | $s_4$      |
| $s_3$         | anything but '\\n'               | $s_{\rm err}$      |
| $s_4$         | any input from $I$  | $s_4$      |
| $s_{err}$     | any input from $I$         | $s_{err}$  |

* The output function $g:S\times I \rightarrow O$ is represented in the figure as the number after the comma on the arrow labels:

| present state | input             | output |
| :-:           | :-:               | :-:        |
| $s_0$         | 'c'               | nothing (e.g., $\lambda$)      |
| $s_0$         | anything but 'c'               | $0$      |
| $s_1$         | 'o'               | nothing (e.g., $\lambda$)     |
| $s_1$         | anything but 'o'               | $0$      |
| $s_2$         | 'w'               | nothing (e.g., $\lambda$)     |
| $s_2$         | anything but 'w'               | $0$      |
| $s_3$         | '\\n'             | $3$     |
| $s_3$         | anything but '\\n'               | $0$      |
| $s_4$         | any input from $I$  | nothing (e.g., $\lambda$)     |
| $s_{err}$     | any input from $I$         | nothing (e.g., $\lambda$) |

There is a reason for outputting 0 or 3 that will be explained later. For now, just think of these outputs as something arbitrary.


___

### How to Implement a FSM in Python -- Approach 1

The problem we want to solve in this notebook is "What is the best way to write Python code that implements a state machine?" There is no perfect "best way", so let's explore. We'll begin with the one that is probably easiest to understand and probably the worst code.

In [36]:
# Define a function that takes the current state and the current input
# and returns the next state and an output.
# This function implements the FSM by a sequence of if then else statements
# The input set, output set, transition function, and output function
# are defined implicitly in the function

# This function takes two strings as input and returns information about
# what the next state and output are. Both the next state and the output
# are returned as strings. The "def" keyword tells Python that you are
# defining a new function.
def finite_state_machine(current_state: str, current_input: str) -> [str,str]:
    
    # These are the two variables that the function will return
    next_state: str
    output: str
    
    # This part implements transitions and outputs from state s0
    if current_state == "s0":
        if current_input == "c": 
            next_state = "s1"
            output = ""
        else:
            next_state = "serr"
            output = "0"
    
    # This part implements transitions and outputs from state s1
    elif current_state == "s1":
        if current_input == "o":
            next_state = "s2"
            output = ""
        else:
            next_state = "serr"
            output = "0"
    
    # This part implements transitions and outputs from state s2
    elif current_state == "s2":
        if current_input == "w":
            next_state = "s3"
            output = ""
        else:
            next_state = "serr"
            output = "0"
    
    # This part implements transitions and outputs from state s3
    elif current_state == "s3":
        if current_input == "\n":
            next_state = "s4"
            output = "3"
        else:
            next_state = "serr"
            output = "0"
    
    # This part implements transitions and outputs from state s3
    elif current_state == "s4":
        next_state = "s4"
        output = ""
    
    # This part implements transitions and outputs from state serr
    else:
        next_state = "serr"
        output = ""
    return next_state, output

Let's define an input string and then trace what happens as we read input characters and run through the state machine.

In [37]:
# Define the input string 
input_string: str = "cow\n"
# Tell the FSM that it should start in state s0
start_state: str = "s0"
print("The start state is " + start_state)

# This variable is used to print the output of the FSM
output: str
# This variable tracks the current_state. The first current state must be the start_state
current_state: str = start_state 

# We'll read one character at a time from the input, beginning at the left.
for character in input_string:
    print("The current state is", current_state)
    print("\tWhen the input is",character)
    # Get the next state and the output from the FSM we defined above
    next_state, output = finite_state_machine(current_state, character)
    print("\tthe next state is", next_state)
    print("\tand the output is",output)
    # Set the current state to the next state so that we can keep running
    current_state = next_state
    print("********************************")

    

The start state is s0
The current state is s0
	When the input is c
	the next state is s1
	and the output is 
********************************
The current state is s1
	When the input is o
	the next state is s2
	and the output is 
********************************
The current state is s2
	When the input is w
	the next state is s3
	and the output is 
********************************
The current state is s3
	When the input is 

	the next state is s4
	and the output is 3
********************************


You should see that characters from the input string are read one by one. Each time a character is read, we advance to a new state and generate an output. Since the input string is "cow\n", we output a bunch of nothings and then a 3. This 3 represents the number of characters successfully read as we ran a FSM designed to see if the input string was "cow".

---

Let's try it on an input other than "cow"

In [24]:
# Define the input string and the start state
input_string: str = "pig\n"
# Tell the FSM that it should start in state s0
start_state: str = "s0"
print("The start state is " + start_state)

# This variable is used to print the output of the FSM
output: str
# This variable tracks the current_state. The first current state must be the start_state
current_state: str = start_state 

# We'll read one character at a time from the input, beginning at the left.
for character in input_string:
    print("The current state is", current_state)
    print("\tWhen the input is",character)
    # Get the next state and the output from the FSM we defined above
    next_state, output = finite_state_machine(current_state, character)
    print("\tthe next state is", next_state)
    print("\tand the output is",output)
    # Set the current state to the next state so that we can keep running
    current_state = next_state
    print("********************************")


The start state is s0
The current state is s0
	When the input is p
	the next state is serr
	and the output is 0
********************************
The current state is serr
	When the input is i
	the next state is serr
	and the output is 
********************************
The current state is serr
	When the input is g
	the next state is serr
	and the output is 
********************************
The current state is serr
	When the input is 

	the next state is serr
	and the output is 
********************************


You should see that each input character is read one by one. Each time a character is read, we advance to a new state and generate an output. Since the input string is not "cow", we output a 0 and then a bunch of nothings. This 0 represents the number of characters successfully read as we ran a FSM designed to see if the input string was "cow".

---
___

### How to Implement a FSM in Python -- Approach 2

Implementing a finite state machine as a bunch of if then else statements does not follow good design principles. The code is long, ugly, and very difficult to maintain if even a small change is needed to the finite state machine. For example, a local government entity had a FSM implemented to control one of their web pages. The FSM had been written by a previous employee. It had over a hundred states, written as nested if-then-else statements. None of the new employees dared modify it because they didn't understand what the FSM did. 
___

Let's try a more modular design.

In the new design, let's treat the finite state machine as its own class, and then have the states of the FSM be functions in the class. It feels kind of weird to treat a state as a function since the pictures all seem to say that a state is something that we visit as we read inputs. But notice that we can also think of a state as a type of process, one that reads and input, generates an output, and gives the next state. Thinking about states more like a process makes it easier to understand how we might implement them as functions.

In [1]:
# The "class" keyword tells python that you are definining a new class
# The format is "class class_name:"
class cow_fsm:

    # This function is called when the class is instantiated. 
    # Python calls this "initializer" function when an object is created from a class.
    # In Python, the initializer function is always called ""__init__".
    # If you don't define this function, Python makes one up for you, so
    # you should always define it yourself so that you don't get surprised.
    def __init__(self) -> None:
        # List the set of state labels, represented as strings.
        self.states: set[str] = {"s0", "s1", "s2", "s3", "s4", "serr"}
        
        # Choose which of the states is the start state
        self.start_state: str = "s0"
        
        # Instead of outputting to the screen, we'll write outputs
        # to a class variable, which we'll access with a "getter"
        self.output: str = ""

    # This function implements what happens in state s0. 
    # It appends the output to the class variable "output",
    # and it returns the name of the next state as a string.
    def s0(self, character:str) -> str:
        next_state: str
        # In state s0, if you read a c output nothing and transition to s1
        if character == "c":
            next_state = "s1"
            self.output += "" # What happens in this line is described below.
        else:
            next_state = "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s1. 
    # It returns the name of the next state as a string.
    def s1(self, character:str) -> str:
        next_state: str
        if character == "o":
            next_state = "s2"
            self.output += "" # I'm going to concatenate all the outputs together
        else:
            next_state = "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s2.  
    # It returns the name of the next state as a string.
    def s2(self, character:str) -> str:
        next_state: str
        if character == "w":
            next_state = "s3"
            self.output += "" # I'm going to concatenate all the outputs together
        else:
            next_state = "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s3.  
    # It returns the name of the next state as a string.
    def s3(self, character:str) -> str:
        next_state: str
        if character == "\n":
            next_state = "s4"
            self.output += "3" # I'm going to concatenate all the outputs together
        else:
            next_state = "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s4.  
    # It returns the name of the next state as a string.
    def s4(self, character:str) -> str:
        next_state = "s4"
        self.output += ""
        return next_state
    
    # This function implements what happens in state serr.  
    # It returns the name of the next state as a string.
    def serr(self, character:str) -> str:
        next_state = "serr"
        self.output += ""
        return next_state
    
    # This function is a public "getter" function that returns the output
    def get_output(self) -> str:
        return self.output
    
    # This function is called to make the FSM run.
    def run(self, input_string: str) -> None:
        # Set the current state to the start state
        current_state = self.start_state
        # "Declare" the type of the name of the next state 
        next_state: str
        # Run through each character in the input
        for character in input_string:
            print("Current state is ", current_state)
            # Check current state name and run corresponding function
            if current_state == "s0":
                next_state = self.s0(character)
            elif current_state == "s1":
                next_state = self.s1(character)
            elif current_state == "s2":
                next_state = self.s2(character)
            elif current_state == "s3":
                next_state = self.s3(character)
            elif current_state == "s4":
                next_state = self.s4(character)
            else:
                next_state = self.serr(character)
            print("\tInput is",character)
            print("\tand next_state is", next_state)
            print("***************************")
            
            # Once you've run all that needs to be done in the current state
            # and figured out the name of the next state, change states
            current_state = next_state

The code is very similar to the if-the-else structure, but we've 
captured what happens in each state inside a function. This design
is more modular and makes it easier to maintain the code.

Notice that the output of the state machine wasn't printed to the screen. 
Instead, each output was appended to an output variable via the line

   - self.output += "something"

The class lecture made it seem like the output of a FSM was something that 
was printed to the screen or otherwise "shared" with the world. But there's
no reason that the output has to be printed to the screen. In the code above,
the output is "printed" to a the "self.output" class member variable.

___

Let's run see what happens when we run the fsm above on an input.

In [2]:
# This creates an FSM object called "my_fsm".
my_fsm: cow_fsm =  cow_fsm()

# This calls the run function from the FSM object 
# and tells it that the input is "cow\n"
my_fsm.run("cow\n")

# The output of the FSM was "printed" to a class member variable.
# Get that class member variable and print it out.
print("output is",my_fsm.get_output())

# This removes the object from memory.
del my_fsm

Current state is  s0
	Input is c
	and next_state is s1
***************************
Current state is  s1
	Input is o
	and next_state is s2
***************************
Current state is  s2
	Input is w
	and next_state is s3
***************************
Current state is  s3
	Input is 

	and next_state is s4
***************************
output is 3


The new code, which used an FSM class that defined states as functions,
ran just like the previous if-the-else FSM. The only differences were:
 - The code used the current state to know what function to call
 - The output went to self.output instead of to the screen

Just to confirm, let's run it on another input.

In [27]:
# This creates an FSM object called "my_fsm".
my_fsm: cow_fsm =  cow_fsm()

# This calls the run function from the FSM object 
# and tells it that the input is "pig\n"
my_fsm.run("pig\n")

# The output of the FSM was "printed" to a class variable.
# Get that class variable and print it out.
print("output is",my_fsm.get_output())

# This removes the object from memory.
del my_fsm

Current state is  s0
	Input is p
	and next_state is serr
***************************
Current state is  serr
	Input is i
	and next_state is serr
***************************
Current state is  serr
	Input is g
	and next_state is serr
***************************
Current state is  serr
	Input is 

	and next_state is serr
***************************
output is 0


The code behaved just like we wanted. 
When the input wasn't "cow", it output a "0'.
___
___

### What is a Higher Order Function?

Python has a neat feature: it treats functions just like variables. Some of you have used these in CS 110 or CS 111. For those who haven't seen these, or as a review for those who have, consider the following example.

In [28]:
def my_function(my_string:str) -> None:
    print(my_string)

Let's do two things with the code:
 - run it
 - print it


In [29]:
# Run it
my_function("hello world")

hello world


Running the code does what we expect. It prints the string we pass to the function.

Now, let's print the function.

In [30]:
# Print it
print(my_function)

<function my_function at 0x128b21d00>


Printing the code shows something like
```
    <function my_function at 0x1280ca0c0>
```
This information about the function itself. Stated simply, 
the information about the function says that the function
is a special kind of object. Just like any other object, it
can be created, passed around, and used.

So, in Python functions are treated just like any other object. An object has
 - a type, which is given by _function_ in the output of the print statement
 - a name, which is given by _my_function_ in the output
 - a memory location, which is given by _0x128ca0c0_ in the output.

Your memory location will be different.

Because functions are just like any other object, we can pass them as
arguments and return them from other functions.

Let's try it.

In [3]:
# The following defines two objects: a string and a function
my_string: str = "my name"
def my_function(my_string: str) -> None:
    print("Inside \"my_function\"")
    print(my_string)

# Define another function
def other_function(argument):
    print("Inside \"other function\"")
    print("\t",argument)

# Call the other function and pass the objects
other_function(my_string)
other_function(my_function)

Inside "other function"
	 my name
Inside "other function"
	 <function my_function at 0x10461fce0>


I can pass the function just like I passed the string.
I can execute the function ...

In [6]:
my_function("hello world")

Inside "my_function"
hello world


... but I can't execute the string object

In [7]:
my_string()

TypeError: 'str' object is not callable

Calling _my_function_ cause the function to execute. 
Trying to call _my_string_ created an error.

When we allow functions to be passed as arguments to other functions
or to be returned from other functions, we are using 
what is called _higher order functions_. We can take advantage of
this and make our FSM code more simple.

___
___
### How to Implement a FSM in Python -- Approach 3

__A more "Pythonic" FSM using higher order functions__

Let's treat our states as functions like before, but let's
omit all the stuff that used strings to identify the function
names. After all, if functions are just objects, we should
be able to use the information about the function directly
without adding the intermediate step of using strings as names.

In [4]:
# The following allows me to return functions from other functions.
from typing import Callable as function

# This is the modified FSM definition.
class cow_fsm:
    # This function is called when the class is instantiated. 
    def __init__(self) -> None:
        ### The old version that used strings 
        #self.states: set[str] = {"s0", "s1", "s2", "s3", "s4", "serr"}
        #self.start_state: str = "s0"
        #self.output: str = ""

        ### The new version that directly uses functions
        self.states: set[function] = {self.s0, self.s1, self.s2, self.s3, self.s4, self.serr}
        self.start_state: function = self.s0
        self.output: str = ""
    
    # This function implements what happens in state s0. It returns the next state as a function.
    def s0(self, character:str) -> function:
        next_state: function
        # In state s0, if you read a c output nothing and return the next
        # state (which is s1) as the function self.s1
        if character == "c":
            next_state = self.s1  # Old version: return "s1"
            self.output += "" # What happens in this line is described below.
        else:
            next_state = self.serr # Old version: return "s1=err"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s1. It returns the next state.
    def s1(self, character:str) -> function:
        next_state: str
        if character == "o":
            next_state = self.s2 # Old version: return "s2"
            self.output += "" 
        else:
            next_state = self.serr # Old version: return "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s2. It returns the next state.
    def s2(self, character:str) -> function:
        next_state: function
        if character == "w":
            next_state = self.s3 # Old version: return "s3"
            self.output += ""
        else:
            next_state = self.serr # Old version: return "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s3. It returns the next state.
    def s3(self, character:str) -> function:
        next_state: function
        if character == "\n":
            next_state = self.s4 # Old version: return "s4"
            self.output += "3"
        else:
            next_state = self.serr # Old version: return "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s4. It returns the next state.
    def s4(self, character:str) -> function:
        next_state: function = self.s4 # Old version: return "s4"
        self.output += ""
        return next_state
    
    # This function implements what happens in state serr. It returns the next state.
    def serr(self, character:str) -> function:
        next_state: function = self.serr # Old version: return "serr"
        self.output += ""
        return next_state
    
    # This function is a public getter function that returns the output
    def get_output(self) -> str:
        return self.output
    
    # This function is called to make the FSM run.
    def run(self, input_string: str) -> None:

        # Set the current state to the start state
        current_state = self.start_state
        
        # "Declare" the type of the next state variable
        next_state: function  # old version next_state: str
        
        # Run through each character in the input
        for character in input_string:
            print("Current state is ", current_state)

            # Check the current state and run the corresponding state function
            if current_state == self.s0: # Old version: current_state == "s0":
                next_state = self.s0(character)
            elif current_state == self.s1: # Old version: current_state == "s1":
                next_state = self.s1(character)
            elif current_state == self.s2: # Old version: current_state == "s2":
                next_state = self.s2(character)
            elif current_state == self.s3: # Old version: current_state == "s3":
                next_state = self.s3(character)
            elif current_state == self.s4: # Old version: current_state == "s4":
                next_state = self.s4(character)
            else:
                next_state = self.serr(character)
            print("\tInput is",character)
            print("\tand next_state is", next_state)
            print("***************************")
            
            # Once you've run all that needs to be done in the current state
            # and figured out the name of the next state, change states
            current_state = next_state

Let's run it

In [5]:
# This creates an FSM object called "my_fsm".
my_fsm: cow_fsm =  cow_fsm()

# This calls the run function from the FSM object 
# and tells it that the input is "cow\n"
my_fsm.run("cow\n")

# The output of the FSM was "printed" to a class variable.
# Get that class variable and print it out.
print("output is",my_fsm.get_output())

# This removes the object from memory.
del my_fsm

Current state is  <bound method cow_fsm.s0 of <__main__.cow_fsm object at 0x104695390>>
	Input is c
	and next_state is <bound method cow_fsm.s1 of <__main__.cow_fsm object at 0x104695390>>
***************************
Current state is  <bound method cow_fsm.s1 of <__main__.cow_fsm object at 0x104695390>>
	Input is o
	and next_state is <bound method cow_fsm.s2 of <__main__.cow_fsm object at 0x104695390>>
***************************
Current state is  <bound method cow_fsm.s2 of <__main__.cow_fsm object at 0x104695390>>
	Input is w
	and next_state is <bound method cow_fsm.s3 of <__main__.cow_fsm object at 0x104695390>>
***************************
Current state is  <bound method cow_fsm.s3 of <__main__.cow_fsm object at 0x104695390>>
	Input is 

	and next_state is <bound method cow_fsm.s4 of <__main__.cow_fsm object at 0x104695390>>
***************************
output is 3


There is more description of the type of the function, probably something
like _< bound method ...>_, but that's just another way of saying it's a
function, one that belongs to another object. Otherwise, it behaves the same way.

___

It seems like a waste of time to first check to see what the state
returned and then turn around and execute it, which is what the
following code does.

```
    if current_state == self.s0: # Old version: current_state == "s0":
        next_state = self.s0(character)
    current_state = next_state
```

Let's replace it with something that executes the return state directly.
It will look more like

```
    current_state = current_state(character)
```

Here's the new code. The only thing that has changed is the run function.

In [8]:
# The following allows me to specify argumetns and return value
# types as functions. In other words, it allows us to follow the
# Python style guide.
from typing import Callable as function

class cow_fsm_final:
    # This function is called when the class is instantiated. 
    # Python calls this "initializer" function when an object is created from a class.
    def __init__(self) -> None:
        self.states: set[function] = {self.s0, self.s1, self.s2, self.s3, self.s4, self.serr}
        self.start_state: function = self.s0
        self.output: str = ""
    
    # This function implements what happens in state s0. It returns the next state as a function.
    def s0(self, character:str) -> function:
        next_state: function
        # In state s0, if you read a c output nothing and transition to s1
        if character == "c":
            next_state = self.s1  # Old version: return "s1"
            self.output += "" 
        else:
            next_state = self.serr # Old version: return "s1=err"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s1. It returns the next state.
    def s1(self, character:str) -> function:
        next_state: str
        if character == "o":
            next_state = self.s2 # Old version: return "s2"
            self.output += "" 
        else:
            next_state = self.serr # Old version: return "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s2. It returns the next state.
    def s2(self, character:str) -> str:
        next_state: str
        if character == "w":
            next_state = self.s3 # Old version: return "s3"
            self.output += "" # I'm going to concatenate all the outputs together
        else:
            next_state = self.serr # Old version: return "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s3. It returns the next state.
    def s3(self, character:str) -> str:
        next_state: str
        if character == "\n":
            next_state = self.s4 # Old version: return "s4"
            self.output += "3" # I'm going to concatenate all the outputs together
        else:
            next_state = self.serr # Old version: return "serr"
            self.output += "0"
        return next_state
    
    # This function implements what happens in state s4. It returns the next state.
    def s4(self, character:str) -> str:
        next_state = self.s4 # Old version: return "s4"
        self.output += ""
        return next_state
    
    # This function implements what happens in state serr. It returns the next state.
    def serr(self, character:str) -> str:
        next_state = self.serr # Old version: return "serr"
        self.output += ""
        return next_state
    
    # This function is a public function that returns the output
    def get_output(self) -> str:
        return self.output
    
    # This function is called to make the FSM run.
    def run(self, input_string: str) -> None:
        # Set the current state to the start state
        current_state = self.start_state
        for character in input_string:
            print("Current state is ", current_state)
            current_state = current_state(character)
            print("\tInput is",character)
            print("\tand next_state is", current_state)
            print("***************************")

Let's run it

In [10]:
# This creates an FSM object called "my_fsm".
my_fsm: cow_fsm_final =  cow_fsm_final()

# This calls the run function from the FSM object 
# and tells it that the input is "cow\n"
my_fsm.run("cow\n")

# The output of the FSM was "printed" to a class variable.
# Get that class variable and print it out.
print("output is",my_fsm.get_output())

# This removes the object from memory.
del my_fsm

Current state is  <bound method cow_fsm_final.s0 of <__main__.cow_fsm_final object at 0x104589a10>>
	Input is c
	and next_state is <bound method cow_fsm_final.s1 of <__main__.cow_fsm_final object at 0x104589a10>>
***************************
Current state is  <bound method cow_fsm_final.s1 of <__main__.cow_fsm_final object at 0x104589a10>>
	Input is o
	and next_state is <bound method cow_fsm_final.s2 of <__main__.cow_fsm_final object at 0x104589a10>>
***************************
Current state is  <bound method cow_fsm_final.s2 of <__main__.cow_fsm_final object at 0x104589a10>>
	Input is w
	and next_state is <bound method cow_fsm_final.s3 of <__main__.cow_fsm_final object at 0x104589a10>>
***************************
Current state is  <bound method cow_fsm_final.s3 of <__main__.cow_fsm_final object at 0x104589a10>>
	Input is 

	and next_state is <bound method cow_fsm_final.s4 of <__main__.cow_fsm_final object at 0x104589a10>>
***************************
output is 3


___

For the homework, you'll implement the following FSM by completing the code below.

![ox FSA](ox_fsm.png)

In [ ]:
from typing import Callable as function

# The following code is not complete. 
class ox_fsm:
    def __init__(self) -> None:
        self.states: set[function] = {self.s0,self.serr} # Complete this
        self.start_state: function = self.s0
        self.output: str = ""
    
    # This function implements what happens in state s0. It returns the next state as a function.
    def s0(self, character:str) -> function:
        next_state: function
        # In state s0, if you read a c output nothing and transition to s1
        if character == "??": # TODO: Fix this
            next_state = self.serr # TODO: Fix this 
            self.output += "" 
        else:
            next_state = self.serr
            self.output += "0"
        return next_state
    
    # TODO: Add whatever other states you need here
    
    # This function implements what happens in state serr. It returns the next state.
    def serr(self, character:str) -> str:
        next_state = self.serr # Old version: return "serr"
        self.output += ""
        return next_state
    
    # This function is a public function that returns the output
    def get_output(self) -> str:
        return self.output
    
    # This function is called to make the FSM run.
    def run(self, input_string: str) -> None:
        # Set the current state to the start state
        current_state = self.start_state
        for character in input_string:
            print("Current state is ", current_state)
            current_state = current_state(character)
            print("\tInput is",character)
            print("\tand next_state is", current_state)
            print("***************************")